In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/201.2 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

from sklearn.model_selection import train_test_split, KFold
from ultralytics import YOLO

In [ ]:
data_dir = r'/content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset'
images_dir = os.path.join(data_dir, 'JPEGImages')
labels_dir = os.path.join(data_dir, 'Labels')

image_files = os.listdir(images_dir)
label_files = os.listdir(labels_dir)

In [ ]:
def move_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        dst_file = os.path.join(dst_dir, file_name)
        try:
            shutil.copy(src_file, dst_file)
        except FileNotFoundError:
            print(f"Archivo no encontrado: {src_file}.")

def create_yaml_content(fold_dir):
    return f"""
            path: {fold_dir}

            train: train/images
            val: val/images

            nc: 2

            names:
                - person
                - car
            """

In [ ]:
train_images_list, test_images = train_test_split(image_files, test_size=0.15, random_state=42)

output_base_dir = r'/content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds'

test_labels=[f.replace('.jpg', '.txt') for f in test_images]

os.makedirs(os.path.join(output_base_dir, r'test/images'), exist_ok=True)
os.makedirs(os.path.join(output_base_dir, r'test/labels'), exist_ok=True)

move_files(test_images, images_dir, os.path.join(output_base_dir, r'test/images'))
move_files(test_labels, labels_dir, os.path.join(output_base_dir, r'test/labels'))

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold_index, (train_index, val_index) in enumerate(kf.split(train_images_list)):

  fold_dir = os.path.join(output_base_dir, f'fold_{fold_index}')

  train_images = os.path.join(fold_dir, r'train/images')
  train_lab = os.path.join(fold_dir, r'train/labels')

  os.makedirs(train_images, exist_ok=True)
  os.makedirs(train_lab, exist_ok=True)

  val_images = os.path.join(fold_dir, r'val/images')
  val_lab = os.path.join(fold_dir, r'val/labels')

  os.makedirs(val_images, exist_ok=True)
  os.makedirs(val_lab, exist_ok=True)

  train_labels = [train_images_list[f].replace('.jpg', '.txt') for f in train_index]
  val_labels=[train_images_list[f].replace('.jpg', '.txt') for f in val_index]

  move_files([train_images_list[i] for i in train_index], images_dir, train_images)
  move_files(train_labels, labels_dir, train_lab)

  move_files([train_images_list[i] for i in val_index], images_dir, val_images)
  move_files(val_labels, labels_dir, val_lab)

  yaml_content = create_yaml_content(fold_dir)

  yaml_path = os.path.join(fold_dir, 'obj_dataset.yaml')
  proyect_results_path = os.path.join(fold_dir, 'training_results')

  with open(yaml_path, 'w') as file:
      file.write(yaml_content)

In [ ]:
  for i in range(5):

    fold_dir = os.path.join(output_base_dir, f'fold_{i}')

    yaml_path = os.path.join(fold_dir, 'obj_dataset.yaml')
    proyect_results_path = os.path.join(fold_dir, 'training_results_150')

    epochs = 150
    weight = 'yolov8n.pt'
    batch = 64

    model = YOLO(weight)
    model.train(data=yaml_path,
                epochs=epochs,
                imgsz=480,
                batch=batch,
                cache=True,
                patience=20,
                project=proyect_results_path,
                name=f'fold_{i}',
                device=0)

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/obj_dataset.yaml, epochs=150, time=None, patience=20, batch=64, imgsz=480, save=True, save_period=-1, cache=True, device=0, workers=8, project=/content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/training_results_150, name=fold_0, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

train: Scanning /content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/train/labels.cache... 684 images, 98 backgrounds, 0 corrupt: 100%|██████████| 684/684 [00:00<?, ?it/s]
train: Caching images (0.2GB RAM): 100%|██████████| 684/684 [00:30<00:00, 22.59it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/val/labels.cache... 171 images, 27 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<?, ?it/s]
val: Caching images (0.1GB RAM): 100%|██████████| 171/171 [00:08<00:00, 20.03it/s]


Plotting labels to /content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/training_results_150/fold_0/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 480 train, 480 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/roadobj dataset-20240707T195100Z-001/roadobj dataset/folds/fold_0/training_results_150/fold_0
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     99/150      5.34G     0.6932     0.4503     0.8683        124        480: 100%|██████████| 11/11 [00:06<00:00,  1.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]

                   all        171        371      0.906      0.905      0.947      0.753



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/150      5.36G     0.7022     0.4489     0.8638        210        480:  82%|████████▏ | 9/11 [00:04<00:01,  1.82it/s]